# work logs

In [ ]:
import json
from pathlib import Path

import jinja2
from IPython.display import HTML, Markdown, display
from nbconvert.filters import ansi2html

In [ ]:
LOGS = sorted(
    (Path.cwd() / "../work/build/logs/").resolve().glob("*.json"),
    key=lambda x: x.stat().st_mtime,
)

In [ ]:
HTML(
    """
<style>
.doit-log { width: 100% !important; }
.doit-log td { text-align: left !important; }
.doit-log th { text-align: left !important; }
.doit-log .elapsed { text-align: right !important; width: 5em;}
.doit-log .result { width: 5em;}
.doit-log th.name { width: 15em;}
details pre {padding: 1em !important;}
</style>
"""
)

In [ ]:
TMPL = jinja2.Template(
    """
<h2>{{ log.stem }}</h2>
<table class="doit-log">
    <thead>
        <tr>
            {% for h in headers %}<th class="{{h}}">{{ h }}</th>{% endfor %}
            {% for f in fields %}<td class="{{f}}">{{ f }}</td>{% endfor %}
            {% for d in details %}<td class="{{d}}">{{ d }}</td>{% endfor %}
        </tr>
    </thead>
    <tbody>
        {% for t in tasks %}
        <tr>
            {% for h in headers %}
            <th >
                {% if h == "name" %}
                {{ t[h].replace("shave:", "") }}
                {% else %}
                {{ t[h] }}
                {% endif %}
            </th>
            {% endfor %}
            {% for f in fields %}<td class="{{ f }}">
                {% if f == "elapsed" and t[f] %}
                    {{ (t[f]) | round(1) }} s
                {% else %}
                    {{ t[f] or "" }}
                {% endif %}
            </td>
            {% endfor %}
            {% for d in details %}<td width="100%">
                {% if t[d] %}
                <details class="jp-RenderedText">
                    <summary>{{ t[d] | count }} lines</summary>
                    <pre>{{ ansi2html("".join(t[d])) }}</pre>
                </details>
                {% endif %}
            </td>{% endfor %}
        </tr>
        {% endfor %}
    </tbody>
</table>
"""
)

In [ ]:
HEADERS = ["result", "name"]
FIELDS = ["elapsed"]
DETAILS = ["out", "err"]
CONTEXT = dict(headers=HEADERS, fields=FIELDS, details=DETAILS, ansi2html=ansi2html)

In [ ]:
for log in LOGS:
    try:
        tasks = [
            t
            for t in json.loads(log.read_text(encoding="utf-8"))["tasks"]
            if t["elapsed"]
        ]
    except:
        continue
    display(HTML(TMPL.render(log=log, tasks=tasks, **CONTEXT)))
else:
    display(Markdown("> No work logs were created"))